In [1]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import image, masking, input_data

In [ ]:


base_dir = '/Users/li/Desktop/debate2025/braindata'

denoised = 'denoised 6'

Schaefer_mask_file = '/Users/li/Desktop/template/Schaefer/tpl-MNI152NLin2009cAsym_res-02_atlas-Schaefer2018_desc-200Parcels7Networks_dseg.nii.gz'
roi_label_file = '/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv'

# 加载模板和ROI标签
Schaefer_mask = nib.load(Schaefer_mask_file)
roi_label = pd.read_csv(roi_label_file)['ROI Name'].values.tolist()

# 查找感兴趣的ROI
desired_roi = '7Networks_LH_Default_PFC_8'
roi_index = roi_label.index(desired_roi) + 1  # Schaefer atlas的索引从1开始

mask_data = Schaefer_mask.get_fdata()
roi_voxel_data = mask_data == roi_index
# 创建被试列表
sub_list = [f'sub-{x:0>3d}' for x in range(13, 51)]
run_list = [1,2,3,4,5,6]
# 遍历每个被试
for sub in sub_list:
    
    new_folder_path = os.path.join(base_dir, denoised, 'ROI Spatial Pattern', desired_roi, sub)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    for run in run_list:
        run_file = f'/Volumes/Li/task-debate/braindata/{denoised}/denoised/{sub}/{sub}_task-debate_run-{run}_denoised_smooth0mm_space-MNI152NLin2009cAsym_res-2_bold.nii.gz'
        run_file = nib.load(run_file)
        run_data = run_file.get_fdata()

        # 提取感兴趣ROI的时间序列
        roi_time_series = run_data[roi_voxel_data]

        # 保存时间序列数据，可以根据需要进行进一步处理
        output_file = os.path.join(new_folder_path, f'{sub}_run-{run}_{desired_roi}_voxel_time_series.npy')
        np.save(output_file, roi_time_series)
        # print(roi_time_series.shape, sep = '\t')
        # print(f'Subject {sub}, Run {run}: ROI time series saved to {output_file}')

    print(f'{sub} Done!')
    print('-----------------------------------------------')
